<a href="https://colab.research.google.com/github/hyakuroume/-/blob/main/%E7%AC%AC2%E7%AB%A0OpenAI%E3%81%AE%E3%83%81%E3%83%A3%E3%83%83%E3%83%88API%E3%81%AE%E5%9F%BA%E7%A4%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('openai_key')

In [ ]:
!pip install openai==1.40.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
client

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role":"system", "content" : "You are a helpful assistant."},
        {"role":"user", "content":"こんにちは!私はジョンと言います！",},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-AgoDRxxy8N8bzcUeuTxubFWLIjD3C",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日は何かお手伝いできることがありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1734766169,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0aa8d3e20b",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 25,
    "total_tokens": 54,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [ ]:
# 会話履歴を踏まえた解答
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"You are a helpful assistant."},
        {"role":"user", "content":"こんにちは！私はジョンと言います！"},
        {"role":"assistant", "content":"こんにちはジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか?"},
        {"role":"user", "content":"私の名前がわかりますか?"},
    ],
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-AgoJ5dr9S2O2IP2OunyvZSjijovLE",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前はジョンです。何か特別なことについてお話ししたいですか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1734766519,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0aa8d3e20b",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 68,
    "total_tokens": 94,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [ ]:
#ストリーミングで応答を得る
#import time

response=client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [
        {"role": "system", "content":"You are a helpful assistant."},
        {"role":"user", "content":"こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)
    #time.sleep(1)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお手伝いできるでしょうか？

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role":"system", "content":"人物一覧を次のJSON形式で出力してください。\n{'people'}:['aaa', 'bbb']}",},
        {"role":"user", "content":"昔々あるところにおじいさんとおばあさんがいました",},
    ],
    response_format = {"type": "json_object"}
)

print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


## Function Calling

In [ ]:
import json

# 関数の定義
def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location":"Tokyo", "temperature":"10", "unit":unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location":"San Francisco", "temperature":"72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location":"Paris", "temperature":"22", "unit":unit})
    else:
        return json.dumps({"location":location, "temperature":"unknown"})

In [ ]:
tools = [
    {
        "type":"function",
        "function":{
            "name":"get_current_weather",
            "description":"Get the current weather in a given location",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"The city and state, e.g. San Francisco, CA",
                    },
                    "unit":{"type":"string", "enum":["celsius", "fahrenheit"]},
                },
                "required":["location"],
            },
        },
    }
]

In [ ]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role":"user", "content":"東京の天気はどうですか?"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,

)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-AgsK0yoD9t2R7mGx9YT9zI4qkmrMk",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_CikxbXASd5G5d4aWdNMRF0C8",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1734781952,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_d28bcae782",
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 81,
    "total_tokens": 104,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": 

In [ ]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [ ]:
available_functions = {
    "get_current_weather":get_current_weather,
}

# 使いたい関数は複数あるかもしれないためループ
for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    #関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id":tool_call.id,
            "role":"tool",
            "name":function_name,
            "content":function_response,
        }
    )

In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_CikxbXASd5G5d4aWdNMRF0C8",
        "function": {
          "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_CikxbXASd5G5d4aWdNMRF0C8",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": \"celsius\"}"
  }
]


In [ ]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,

)

print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-AgsVD10TfXjIO5YFciHrWREP1bR2w",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "東京の現在の気温は約10℃です。天気について詳しい情報が必要であれば、具体的な天気予報サイトなどを確認してください。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1734782647,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_d28bcae782",
  "usage": {
    "completion_tokens": 38,
    "prompt_tokens": 66,
    "total_tokens": 104,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
